# Initial Data Analysis - Capstone Project

In [110]:
# Imports
import os
import csv
import numpy as np
import pandas as pd
from pandas import Int64Index
import pickle
import sys

In [111]:
def save_as_pkl(object, path):
	pickle.dump(object, open(path, "wb"))

def load_pkl(path):
	obj = pickle.load(open(path, "rb"))
	return obj

### Preprocessing and loading the data

In [112]:
# Processing the .csvs into dataframes and saving them as pickles for easier
# loading on future runs
files = [f for f in os.listdir("data/") if f.split('1')[0] == 'file']
for file in files:
    print ('\n'+file)
    fName = file.split('.')[0]
    df = pd.read_csv('data/' + file, delimiter=',', na_values=['NA'])
#     df["Id"] = df.reset_index().index
#     df.set_index("Id")
    print(df.shape)
    save_as_pkl(df, 'pickles/'+fName+'.pkl')


file10.csv
(3979, 392)

file11.csv
(3958, 392)

file12.csv
(4286, 392)

file13.csv
(4638, 392)

file14.csv
(4836, 392)

file15.csv
(5557, 392)

file16.csv
(5456, 392)

file17.csv
(5207, 392)


In [113]:
## Loading preprocessed dataframes
pklFiles = [f for f in os.listdir("pickles/") if f.split('1')[0] == 'file']
for file in pklFiles:
	df = load_pkl("pickles/" + file)
	print(df.shape)

(3979, 392)
(3958, 392)
(4286, 392)
(4638, 392)
(4836, 392)
(5557, 392)
(5456, 392)
(5207, 392)


# Change Year Loaded Here:

In [134]:
# YEAR:
fname = "file17.pkl"
df = load_pkl("pickles/"+fname)

### Starting to analyze the data

In [135]:
# First 30 columns:
firstPart = df.iloc[:,0:30]
# print(firstPart.columns.values)

# Columns 30 to 65: (top 6 course marks)
top6CourseMarks = df.iloc[:,30:66]
# print(top6CourseMarks.columns.values)

# cols = []
# for column in top6CourseMarks[top6CourseMarks.columns[2::3]]:
#     cols.append(df[column])
# hstack = pd.concat([x for x in cols], axis=1)
# hstack.fillna(0, inplace=True)
# hstack['Sum'] = hstack.mean(axis=1)
# print(hstack)

# Columns 66 to 71:
middle = df.iloc[:,66:72]
print(middle["WAVERG1"], middle["WAVERG2"])
# print(middle.columns.values)

# Columns 72 to 391:
choices = df.iloc[:, 72:]
# print(choices.columns.values)

0         0
1         0
2         0
3         0
4         0
       ... 
5202    813
5203    820
5204      0
5205      0
5206      0
Name: WAVERG1, Length: 5207, dtype: int64 0       895
1       833
2       830
3       965
4       913
       ... 
5202    813
5203    820
5204    905
5205      0
5206    890
Name: WAVERG2, Length: 5207, dtype: int64


In [136]:
# # COlumns 72 to 391:
# choices = df.iloc[:, 72:]
# print(choices.columns.values)
# for col in choices.columns.values:
#     print[]
    
# cols = []
# for column in top6CourseMarks[top6CourseMarks.columns[2::3]]:
#     cols.append(df[column])
# hstack = pd.concat([x for x in cols], axis=1)
# # hstack.fillna(0, inplace=True)
# hstack['Sum'] = hstack.mean(axis=1)
# print(hstack)


# TODOS:
- Determine the cutoff by finding lowest GPA student that got into Mac program
- Group choices for each student
- Divide dataset into those who got into Mac, and those who did not.

# Determining grade cutoff

### Seperating relevant data
- Splitting data into students who accept and offer from mac and those who don't.
- Isolating 8 initial columns of interest for analysis

In [137]:
## Starting columns of interest:
# RESPROV - Province of Residence
# RESCNTY - County of Residence
# ZIP3 - Residence Postal Code (First 3 Digits)
# CONFUNI - Confirmed University (OurUni='196')
# CONFPR - Confirmed Program (OurProg='SI', OurProg_coop='SIA')
# CONFCHOIC - OUAC Confirmed Choice Preference
# WAVERG1 - Weighted Average (best 6 OAC / Senior Level current year finals)
# WAVERG2 - Weighted Average (best 6 OAC / Senior Level all year finals)
## First 6 columns are in first half, last 2 are in "Middle" dataframe.

COIsFirstHalf = ["RESPROV", "RESCNTY", "ZIP3", "CONFUNI", "CONFPR", "CONFCHOIC"]
COIsSecondHalf = ["WAVERG1", "WAVERG2"]

# Adding columns of interest found in first half of the data
columnsOfInterest = pd.concat([firstPart[x] for x in COIsFirstHalf], axis=1)

# Adding the rest of the columns of interest
columnsOfInterest = pd.concat([columnsOfInterest] + [middle[x] for x in COIsSecondHalf],axis=1)
copy = columnsOfInterest.copy()

mcmasterVector = copy["CONFUNI"] == 196
notmcmasterVector = copy["CONFUNI"] != 196

### Some min/max/average output for both student types:

In [139]:
# Students that received and accepted a McMaster offer:
acceptedMcMaster = copy[mcmasterVector]
# print(acceptedMcMaster)
# print(acceptedMcMaster[(acceptedMcMaster["WAVERG2"] > 0) & (acceptedMcMaster["CONFPR"] == "SIA")].min())
print("Students who accepted an offer from McMaster:")
print("Mean values for each column:")
print(acceptedMcMaster[(acceptedMcMaster["WAVERG2"] > 0) & (acceptedMcMaster["CONFPR"] == "SIA")].mean())
print("\nMax values for each column:")
print(acceptedMcMaster[(acceptedMcMaster["WAVERG2"] > 0) & (acceptedMcMaster["CONFPR"] == "SIA")].max())
print("\nLowest 5 weighted average:")
print(acceptedMcMaster[(acceptedMcMaster["WAVERG2"] > 0) & (acceptedMcMaster["CONFPR"] == "SI")].nsmallest(10,"WAVERG2"))
print("\nShape of acceptedMcMaster df (number of students that accepted):")
print(acceptedMcMaster.shape)

# Students that didn't accept a mcmaster offer:
print('\n\n\n\n')
notMcMaster = copy[notmcmasterVector]
# print(notMcMaster)
print("Min average of students who didn't accept mac offer")
print(notMcMaster[notMcMaster["WAVERG2"] > 0].min())
print("\nMax average of students who didn't accept mac offer")
print(notMcMaster[notMcMaster["WAVERG2"] > 0].max())
print("\nAverage average of students who didn't accept mac offer")
print(notMcMaster[notMcMaster["WAVERG2"] > 0].mean())


Students who accepted an offer from McMaster:
Mean values for each column:
RESPROV      124.922652
RESCNTY      148.733425
CONFUNI      196.000000
CONFCHOIC      2.490331
WAVERG1      745.044199
WAVERG2      911.273481
dtype: float64

Max values for each column:
RESPROV      125
RESCNTY      185
CONFUNI      196
CONFPR       SIA
CONFCHOIC     10
WAVERG1      990
WAVERG2      990
dtype: object

Lowest 5 weighted average:
      RESPROV  RESCNTY ZIP3  CONFUNI CONFPR  CONFCHOIC  WAVERG1  WAVERG2
2785      125      134  M2U    196.0     SI        2.0      850      850
2797      125      146  M7N    196.0     SI        2.0      852      852
3878      125      143  N3I    196.0     SI        1.0        0      858
1858      125      185  L2T    196.0     SI        5.0      867      867
3526      125      143  N0D    196.0     SI        3.0      867      867
4002      125      143  N7Q    196.0     SI        2.0      868      868
4928      125      164  M0H    196.0     SI        1.0      825  